In [2]:
import os 
os.environ['OMP_NUM_THREADS'] = '1'

In [39]:
from mindquantum import H, Hamiltonian, X, Z, RX, RY, RZ, Circuit
import mindquantum as mq
import numpy as np
import mindspore as ms
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")

In [8]:
H.matrix()

array([[ 0.70710678,  0.70710678],
       [ 0.70710678, -0.70710678]])

In [9]:
circ = Circuit([H.on(0), X.on(1, 0)])
circ

q0: ──H────●──
           │  
q1: ───────X──

In [10]:
circ += mq.qft([0, 1, 2])
circ

q0: ──H────●────H────PS(π/2)────PS(π/4)─────────────────────────@──
           │            │          │                            │  
q1: ───────X────────────●──────────┼───────H────PS(π/2)─────────┼──
                                   │               │            │  
q2: ───────────────────────────────●───────────────●───────H────@──

In [12]:
circ += mq.ZZ('theta').on([0, 1])

In [13]:
circ

q0: ──H────●────H────PS(π/2)────PS(π/4)─────────────────────────@────ZZ(theta)──
           │            │          │                            │        │      
q1: ───────X────────────●──────────┼───────H────PS(π/2)─────────┼────ZZ(theta)──
                                   │               │            │               
q2: ───────────────────────────────●───────────────●───────H────@───────────────

In [25]:
sim = mq.Simulator('projectq', 3)

In [26]:
circ = Circuit()
for i in range(3):
    circ += mq.H(i)

circ += mq.qft(range(3))
sim.apply_circuit(circ)
sim

projectq simulator with 3 qubits.
Current quantum state:
1¦000⟩

In [5]:
enconder = Circuit()
enconder += H.on(0)
enconder += RX('alpha0').on(0)
enconder += RY('alpha1').on(0)
enconder += RZ('alpha2').on(0)

In [8]:
enconder.get_qs('projectq', {'alpha0': 0.1, 'alpha1': 0.2, 'alpha2': 0.3}, ket=True)

'(0.6203638230723639-0.1257539726032947j)¦0⟩\n(0.7702994677056821+0.07728774449984671j)¦1⟩'

In [9]:
ansatz = Circuit()
ansatz += RX('theta0').on(0)
ansatz += RY('theta1').on(0)

In [12]:
circ = enconder + ansatz
circ

q0: ──H────RX(alpha0)────RY(alpha1)────RZ(alpha2)────RX(theta0)────RY(theta1)──

In [14]:
sim = mq.Simulator('projectq', circ.n_qubits)
sim

projectq simulator with 1 qubit.
Current quantum state:
1¦0⟩

In [18]:
ham = Hamiltonian(-mq.QubitOperator('Z0'))

In [20]:
grad_ops = sim.get_expectation_with_grad(ham, enconder, ansatz, enconder.params_name, ansatz.params_name)

In [41]:
encoder_init = np.random.rand(1, len(enconder.params_name))
ansatz_init = np.random.rand(len(ansatz.params_name))
# grad_op = grad_ops(encoder_init, ansatz_init)
net = mq.MQLayer(grad_ops)

In [45]:
net(ms.Tensor([[0.1, 0.2, 0.3]]))

Tensor(shape=[1, 1], dtype=Float32, value=
[[-6.19833112e-01]])

In [53]:
opt = ms.nn.Adam(net.trainable_params(), learning_rate=0.5)
train_net = ms.nn.TrainOneStepCell(net, opt)

In [54]:
train_net

TrainOneStepCell<
  (network): MQLayer<
    (evolution): MQOps<1 qubit projectq VQA Operator>
    >
  (optimizer): Adam<>
  >

In [55]:
for i in range(100):
    res = train_net(ms.Tensor(np.array([[0.1, 0.2, 0.3]])))
    print(res)

[[-0.6198331]]
[[-0.7929457]]
[[-0.9184655]]
[[-0.9859705]]
[[-0.99490654]]
[[-0.9735479]]
[[-0.94153917]]
[[-0.9170432]]
[[-0.9121312]]
[[-0.92504424]]
[[-0.9463014]]
[[-0.9681088]]
[[-0.9865593]]
[[-0.99784446]]
[[-0.99915946]]
[[-0.9925711]]
[[-0.9835679]]
[[-0.9761919]]
[[-0.97220445]]
[[-0.9727165]]
[[-0.9778934]]
[[-0.98565084]]
[[-0.9928147]]
[[-0.99752086]]
[[-0.9994803]]
[[-0.9987216]]
[[-0.9957717]]
[[-0.99233735]]
[[-0.9902595]]
[[-0.9901044]]
[[-0.9914919]]
[[-0.9939708]]
[[-0.99684227]]
[[-0.99898946]]
[[-0.9997265]]
[[-0.99929786]]
[[-0.9982482]]
[[-0.99701184]]
[[-0.9961534]]
[[-0.9961627]]
[[-0.9969395]]
[[-0.99800384]]
[[-0.9990057]]
[[-0.9996858]]
[[-0.99979824]]
[[-0.9994089]]
[[-0.99888426]]
[[-0.99849737]]
[[-0.9983622]]
[[-0.99856085]]
[[-0.9990152]]
[[-0.99946654]]
[[-0.9997395]]
[[-0.99980867]]
[[-0.9996801]]
[[-0.99944115]]
[[-0.9992656]]
[[-0.9992399]]
[[-0.9993389]]
[[-0.9995231]]
[[-0.99971557]]
[[-0.9998155]]
[[-0.9998054]]
[[-0.99973226]]
[[-0.99963766]]
[

In [57]:
net.trainable_params()[0].asnumpy()

array([ 0.2952324, -1.7948754], dtype=float32)